In [1]:
import pandas as pd
pd.__version__

'0.23.4'

In [53]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
from sklearn import model_selection, pipeline, linear_model, metrics, preprocessing
from sklearn.tree import DecisionTreeClassifier

In [52]:
df = pd.read_csv("train_data_titanic.csv")
df.head()

,Unnamed: 0,PassengerId,Survived,Sex,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Family_size,Title_1,Title_2,Title_3,Title_4,Emb_1,Emb_2,Emb_3
0,0,1,0,1,0.2750,0.014151,0,0,1,0.1,1,0,0,0,0,0,1
1,1,2,1,0,0.4750,0.139136,1,0,0,0.1,1,0,0,0,1,0,0
2,2,3,1,0,0.3250,0.015469,0,0,1,0.0,0,0,0,1,0,0,1
3,3,4,1,0,0.4375,0.103644,1,0,0,0.1,1,0,0,0,0,0,1
4,4,5,0,1,0.4375,0.015713,0,0,1,0.0,1,0,0,0,0,0,1


In [4]:
#df.describe()

In [5]:
y = df.Survived
x = df.drop(columns="Survived")
#y.head(), x.head()

In [96]:
xtrain, xtest, ytrain, ytest = model_selection.train_test_split(x.values, y.values, random_state=1, test_size=0.3)

In [175]:
%%time
pipe = pipeline.Pipeline([
    #"""
    #                ("model", linear_model.SGDClassifier(max_iter=1e3,
    #                                                      tol=1e-3,
    #                                                      eta0=0.2,
    #                                                      learning_rate="invscaling",
    #                                                      alpha=0.1,
    #                                                      random_state=1,
    #                                                      l1_ratio=0.9)
    #                )
    #"""
                    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
                    ("scale", preprocessing.StandardScaler()),
                    ("model", linear_model.LogisticRegression(solver="liblinear", random_state=1))
                    ])

CPU times: user 228 µs, sys: 3 µs, total: 231 µs
Wall time: 261 µs


In [176]:
%%time
pipe.fit(xtrain, ytrain)

CPU times: user 7.22 ms, sys: 1.98 ms, total: 9.19 ms
Wall time: 8.85 ms


Pipeline(memory=None,
     steps=[('poly', PolynomialFeatures(degree=1, include_bias=False, interaction_only=False)), ('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('model', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=1, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False))])

In [177]:
%%time
ytrain_pred = pipe.predict(xtrain)
ytest_pred = pipe.predict(xtest)

CPU times: user 3.6 ms, sys: 2.33 ms, total: 5.93 ms
Wall time: 3.97 ms


In [172]:
est = pipe.steps[-1][-1]
est.n_iter_, ytest_pred.sum()

(array([6], dtype=int32), 89)

In [178]:
%%time
metrics.confusion_matrix(ytest, ytest_pred)

CPU times: user 2.44 ms, sys: 1.35 ms, total: 3.78 ms
Wall time: 2.61 ms


array([[118,  24],
       [ 31,  65]])

In [174]:
print("accuracy train : ", metrics.accuracy_score(ytrain, ytrain_pred))
print("accuracy test  : ", metrics.accuracy_score(ytest , ytest_pred ))
print('')
print("precision train : ", metrics.precision_score(ytrain, ytrain_pred))
print("precision test  : ", metrics.precision_score(ytest , ytest_pred ))
print('')
print("recall train : ", metrics.recall_score(ytrain, ytrain_pred))
print("recall test  : ", metrics.recall_score(ytest , ytest_pred ))

accuracy train :  0.8483754512635379
accuracy test  :  0.7689075630252101

precision train :  0.8214285714285714
precision test  :  0.7303370786516854

recall train :  0.7666666666666667
recall test  :  0.6770833333333334


In [148]:
#pd.DataFrame({"actual":ytest, "prediction":ytest_pred})
#pd.DataFrame({"feature": x.columns, "coeff": est.coef_[0]})